In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt

In [2]:
#Create dataframe from data
data=pd.read_csv('Sales.csv')
data = data.astype(str)
data.Product = 'PN' + data.Product.astype(str)
data['isProduct']=data['Product'].apply(lambda x:len(x))

data=data[data['isProduct']==15]#Only SKU to evaluate

data.drop(data.columns[[1,3,4,5,6,-1]],axis=1,inplace=True)#Drop columns not usable
#Fields formatting
data.PostingDate=pd.to_datetime(data['PostingDate'], format='%Y-%m-%d')
data.Quantity=pd.to_numeric(data['Quantity'],downcast='integer')
data.TotalSale=pd.to_numeric(data['TotalSale'],downcast='float')

data=data[data['Quantity']>0].reset_index().rename(columns={'Product': "PN"})#Only positive sales
data=data.groupby(['PostingDate','PN']).agg({'Quantity':'sum','TotalSale':'sum'}).reset_index()#in case 2 sales on the same date

In [3]:
# unique SKU
pn2023 = data['PN'].unique()

In [4]:
#unique id per date
dates = pd.date_range('2018-01-01', '2023-12-31')
dates

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08',
               '2018-01-09', '2018-01-10',
               ...
               '2023-12-22', '2023-12-23', '2023-12-24', '2023-12-25',
               '2023-12-26', '2023-12-27', '2023-12-28', '2023-12-29',
               '2023-12-30', '2023-12-31'],
              dtype='datetime64[ns]', length=2191, freq='D')

In [5]:
#Total of rows when making the full outer join
print('Total SKU = ',len(pn2023)) #total SKU
print('Total dates = ',dates.nunique())#total dates
print('Rows= ',len(pn2023)*dates.nunique())#total rows

Total SKU =  198
Total dates =  2191
Rows=  433818


In [6]:
# Dataframe with all rows for all SKU
pn_index=pd.DataFrame(pn2023).rename(columns={0: "PN"})
pn_index.set_index('PN',inplace=True,drop=False)
pn_index=pn_index.apply(
        lambda x: dates, axis=1
    ).explode().reset_index(name='PostingDate')[['PostingDate', 'PN']]
len(pn_index)

433818

In [7]:
#Create full dataframe
df=pn_index.merge(data,how='left', on=['PostingDate','PN'])#left join with full index
df['Quantity'] = df['Quantity'].fillna(0) #in case of null, fill with zero
df['TotalSale'] = df['TotalSale'].fillna(0) #in case of null, fill with zero

#adding date fields
df['PN10']=df.PN.str[:12] #Trim the SKU
df['Year']=df.PostingDate.dt.year
df['Month']=df.PostingDate.dt.month
df['Week']=df.PostingDate.dt.isocalendar().week
df['DayofWeek']=df.PostingDate.dt.dayofweek

print('Posting Date',df['PostingDate'].nunique())
print('SKU length 13',df['PN'].nunique())
print('SKU length 10',df['PN10'].nunique())

Posting Date 2191
SKU length 13 198
SKU length 10 71


In [8]:
# Checking we did not miss data
print(data['Quantity'].sum())
print(df['Quantity'].sum())
print(data['TotalSale'].sum())
print(df['TotalSale'].sum())

87623595
87623595.0
356551170.5872
356551170.5872


In [9]:
df['PN10'].nunique()

71

In [10]:
# Checking if we group by PN10 SKU how many rows we get
print(df['PN10'].nunique())
print(dates.nunique())
print('Rows= ',df['PN10'].nunique()*dates.nunique())

71
2191
Rows=  155561


In [11]:
# write data which will be used
df.to_csv('SalesbyPN.csv')